### REFERENCES
@inproceedings{antoun2020arabert,
  title={AraBERT: Transformer-based Model for Arabic Language Understanding},
  author={Antoun, Wissam and Baly, Fady and Hajj, Hazem},
  booktitle={LREC 2020 Workshop Language Resources and Evaluation Conference 11--16 May 2020},
  pages={9}
}


Arabic language understanding with Bert - Wissam Antoun (2020) YouTube. Available at: https://www.youtube.com/watch?v=N9pQ3CZ0v6U (Accessed: 17 September 2023). 

Khalid, S. (2020) Bert explained: A Complete Guide with Theory and tutorial, Medium. Available at: https://medium.com/@samia.khalid/bert-explained-a-complete-guide-with-theory-and-tutorial-3ac9ebc8fa7c (Accessed: 17 September 2023). 

Classify text with Bert&nbsp; :&nbsp;  text&nbsp; :&nbsp;  tensorflow (no date) TensorFlow. Available at: https://www.tensorflow.org/text/tutorials/classify_text_with_bert (Accessed: 17 September 2023). 

Text classification using Bert &amp; Tensorflow | Deep Learning Tutorial 47 (Tensorflow, Keras &amp; Python) (2021) YouTube. Available at: https://www.youtube.com/watch?v=hOCDJyZ6quA (Accessed: 17 September 2023). 

In [ ]:
!pip install transformers[torch]
!pip install accelerate -U
import accelerate
import transformers
print(accelerate.__file__)
print(transformers.__file__)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 28.5 MB/s eta 0:00:00
/usr/local/lib/python3.10/dist-packages/accelerate/__init__.py
/usr/local/lib/python3.10/dist-packages/transformers/__init__.py


In [ ]:
import sys

print(sys.version)
print(sys.path)
import accelerate
import transformers
print("Accelerate version:", accelerate.__version__)
print("Transformers version:", transformers.__version__)


3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/tmp/tmpxiisbt22']
Accelerate version: 0.22.0
Transformers version: 4.32.1


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv('UseThisClean.csv')
comments = df['clean_text'].tolist()
labels = df['Class'].tolist()

# Split data: 80% training, 20% validation
train_texts, val_texts, train_labels, val_labels = train_test_split(comments, labels, test_size=0.2, random_state=42, stratify=labels)


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

MODEL_NAME = 'aubmindlab/bert-base-arabertv02'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers.models.bert.tokenization_bert import BertTokenizer
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

# Load your specific tokenizer
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# Load the dataset
data = pd.read_csv('UseThisClean.csv')

# Drop NaN values
data = data.dropna(subset=['clean_text'])

# Convert the 'clean_text' column to string
data['clean_text'] = data['clean_text'].astype(str)

# Filter out empty strings
data = data[data['clean_text'].str.strip() != ""]

# Extract the 'clean_text' column and the 'Class' column
texts = data['clean_text'].tolist()
labels = data['Class'].tolist()

# Tokenize the data
inputs = tokenizer(texts, padding=True, truncation=True, max_length=25, return_tensors='pt')

# Extract input_ids and attention masks
input_ids = inputs["input_ids"]
attention_masks = inputs["attention_mask"]

# Convert labels to tensors
label_tensors = torch.tensor(labels)

# Split data into training and validation sets
train_inputs, val_inputs, train_masks, val_masks, train_labels, val_labels = train_test_split(
    input_ids, attention_masks, label_tensors, test_size=0.2
)


In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, classification_report
import torch

# Define the custom dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {'input_ids': self.inputs[idx], 'labels': self.labels[idx]}

# Instantiate the datasets
train_dataset = CustomDataset(train_inputs, train_labels)
eval_dataset = CustomDataset(val_inputs, val_labels)


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./work',  # <-- This line is added. Adjust the path as needed.
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    fp16=True,
    save_total_limit=2,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=200,
    do_train=True,
    do_eval=True,
    dataloader_pin_memory=False,  # Add this line to disable memory pinning
    evaluation_strategy="epoch"
)

from sklearn.metrics import accuracy_score, classification_report

# Define the metric computation
def compute_metrics(p):
    pred_labels = p.predictions.argmax(-1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, pred_labels)
    report = classification_report(labels, pred_labels)
    return {
        'accuracy': accuracy,
        'classification_report': report
    }

# Move input tensors to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_inputs = train_inputs.to(device)
val_inputs = val_inputs.to(device)
train_labels = train_labels.to(device)
val_labels = val_labels.to(device)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

train_inputs = train_inputs.to(device)
val_inputs = val_inputs.to(device)
train_labels = train_labels.to(device)
val_labels = val_labels.to(device)




In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,Classification Report
0,0.255700,0.805850,0.792822,precision recall f1-score support 0 0.80 0.76 0.78 794 1 0.81 0.86 0.84 876 2 0.76 0.75 0.75 782 accuracy 0.79 2452 macro avg 0.79 0.79 0.79 2452 weighted avg 0.79 0.79 0.79 2452
2,0.149200,0.840747,0.806688,precision recall f1-score support 0 0.76 0.84 0.80 794 1 0.84 0.85 0.84 876 2 0.82 0.72 0.77 782 accuracy 0.81 2452 macro avg 0.81 0.80 0.80 2452 weighted avg 0.81 0.81 0.81 2452
2,0.085600,0.976305,0.814029,precision recall f1-score support 0 0.80 0.81 0.81 794 1 0.83 0.87 0.85 876 2 0.81 0.75 0.78 782 accuracy 0.81 2452 macro avg 0.81 0.81 0.81 2452 weighted avg 0.81 0.81 0.81 2452


Trainer is attempting to log a value of "              precision    recall  f1-score   support

           0       0.80      0.76      0.78       794
           1       0.81      0.86      0.84       876
           2       0.76      0.75      0.75       782

    accuracy                           0.79      2452
   macro avg       0.79      0.79      0.79      2452
weighted avg       0.79      0.79      0.79      2452
" of type <class 'str'> for key "eval/classification_report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "              precision    recall  f1-score   support

           0       0.76      0.84      0.80       794
           1       0.84      0.85      0.84       876
           2       0.82      0.72      0.77       782

    accuracy                           0.81      2452
   macro avg       0.81      0.80      0.80      2452
weighted avg       0.81      0.81      0.8

TrainOutput(global_step=918, training_loss=0.15915151982525594, metrics={'train_runtime': 450.3959, 'train_samples_per_second': 65.309, 'train_steps_per_second': 2.038, 'total_flos': 377326224544500.0, 'train_loss': 0.15915151982525594, 'epoch': 3.0})

In [ ]:
results = trainer.evaluate()
print(results)


Trainer is attempting to log a value of "              precision    recall  f1-score   support

           0       0.80      0.81      0.81       794
           1       0.83      0.87      0.85       876
           2       0.81      0.75      0.78       782

    accuracy                           0.81      2452
   macro avg       0.81      0.81      0.81      2452
weighted avg       0.81      0.81      0.81      2452
" of type <class 'str'> for key "eval/classification_report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.9763054251670837, 'eval_accuracy': 0.8140293637846656, 'eval_classification_report': '              precision    recall  f1-score   support\n\n           0       0.80      0.81      0.81       794\n           1       0.83      0.87      0.85       876\n           2       0.81      0.75      0.78       782\n\n    accuracy                           0.81      2452\n   macro avg       0.81      0.81      0.81      2452\nweighted avg       0.81      0.81      0.81      2452\n', 'eval_runtime': 11.211, 'eval_samples_per_second': 218.714, 'eval_steps_per_second': 54.678, 'epoch': 3.0}
